# Movie Review Analysis Outline
## import necessary python libraries


In [1]:


# "transformData.py" is this script which is intended to be run once on our data
# transform the raw data into a rdd readable format first


# import all necessary libraries
import re
import string
from operator import add
import os
import sys
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint

import pyspark.mllib.regression as mllib_reg
import pyspark.mllib.linalg as mllib_lalg
import pyspark.mllib.classification as mllib_class
import pyspark.mllib.tree as mllib_tree



print (pyspark) # test to see that pyspark is up and running okay



<module 'pyspark' from '/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/__init__.py'>


NameError: name 'spark' is not defined

# CLEAN DATA

In [2]:
# get rid of all html tags in the data
def strip_html_tags(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data.lower())

#parse and take care of funky symbols in summary and text <- normalize to root words maybe?
def cleanData (data):
    data = strip_html_tags(data)
    data =  re.sub("[\t\,\:;\(\)\"\'\~\-\!\?\`]", "",data, 0, 0)
    #data =  re.sub("[\.0`]", "",data, 0, 0) # special case to get rid of ".0" of scores -- 
    return data

#re.sub("[\.\t\,\:;\(\)\.]", " ", strip_html_tags(data.lower()), 0, 0)

# clean the data for each element of each sub list
def prepData (list_str):
    L= []
    for x in list_str:
        #print x
        L.append(cleanData(x).strip())
    return L
#####
# Notes 
# headers for the project
# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']



# PARSE THE FILE WE WANT TO USE AS OUR DATA

In [3]:
# transform Data uses /// as a separator for each eleent of data for each movie review
# the original file for the current output.txt is a truncated version of all our data, so the end elements will be funky

# output < smaller_parsed

movies_txt2 = sc.textFile("50k_parsed.txt").map(lambda x: (cleanData(x).split('///')))
#movies_txt2 = sc.textFile("smaller_parsed.txt").map(lambda x: (cleanData(x).split('///')))


## Parsing out relevant fields - score and text of review

In [4]:
# prep our data

#####
# Notes 
# headers for the project
# [u'productId', u'userId', u'profileName', u'helpfulness', u'score', u'time', u'summary', u'text']
# [u'productId', _ , _ , _ , u'score' [0] to get rid of .0, _  u'summary', u'text']

# take the data that we care most about
movies_new = movies_txt2.map(lambda L: (L[0], L[4][0], L[6], L[7]) if len(L) == 8 else L )
movies_new1 = movies_new.filter(lambda L: len(L) == 4) # lets just take all the data that has been parsed correctly

#print (movies_new.top(20))
print (movies_new1.count(), movies_new.count())
print (movies_new.count() == movies_new1.count())

removeHTMLTags = movies_new1.map(prepData) # remove html tags

(49994, 50000)
False


## REVISED STOP WORDS REMOVAL 

In [58]:
#REVISED STOP WORDS REMOVAL Approach

#used the stopwords file in the virtual box instead... taking too long
baseDir = os.path.join('../data')
inputPath = os.path.join('cs100', 'lab3')
STOPWORDS_PATH = 'stopwords.txt'
split_regex = r'\W+'

stopfile = os.path.join(baseDir, inputPath, STOPWORDS_PATH)
stopwords = set(sc.textFile(stopfile).collect())


def tokenize(string):
    #reusing assignment code
    """ An implementation of input string tokenization to exclude stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    splitList = re.split(split_regex, string.lower().strip())
    
    return [x for x in splitList if x!="" and x not in stopwords]  



removeSW_output= removeHTMLTags.map(lambda x: (  int(x[1]), tokenize(x[3]))  )
print removeSW_output.top(1)

[(5, [u'zulu', u'superlative', u'depiction', u'19th', u'century', u'battle', u'rourkes', u'drift', u'british', u'outpost', u'heavily', u'outnumbered', u'seige', u'zulu', u'warriors', u'fought', u'desperately', u'almost', u'last', u'man', u'somehow', u'prevailed', u'terrifying', u'exciting', u'war', u'film', u'unlike', u'splendid', u'cast', u'word', u'diamond', u'entertainment', u'tapesmy', u'experience', u'slp', u'speed', u'vhs', u'videos', u'tracks', u'remarkably', u'well', u'usually', u'recorded', u'hifi', u'sound', u'well', u'standard', u'linear', u'format', u'zulu', u'also', u'available', u'dvd', u'also', u'recommendedzulu', u'dawn', u'1979', u'tells', u'complete', u'story', u'struggles', u'tribesmen', u'british', u'soldiers', u'commenced', u'1879', u'burt', u'lancaster', u'vhs', u'edition', u'dvd', u'editionparenthetical', u'number', u'preceding', u'title', u'1', u'10', u'viewer', u'poll', u'rating', u'found', u'film', u'resource', u'website', u'8', u'0', u'zulu', u'uk1964', u'sta

## Vectorizing - making the vector of features...

In [74]:

removeSW_output= removeHTMLTags.map(lambda x: (  int(x[1]), tokenize(x[3]))  )
print removeSW_output.top(1)

[(5, [u'zulu', u'superlative', u'depiction', u'19th', u'century', u'battle', u'rourkes', u'drift', u'british', u'outpost', u'heavily', u'outnumbered', u'seige', u'zulu', u'warriors', u'fought', u'desperately', u'almost', u'last', u'man', u'somehow', u'prevailed', u'terrifying', u'exciting', u'war', u'film', u'unlike', u'splendid', u'cast', u'word', u'diamond', u'entertainment', u'tapesmy', u'experience', u'slp', u'speed', u'vhs', u'videos', u'tracks', u'remarkably', u'well', u'usually', u'recorded', u'hifi', u'sound', u'well', u'standard', u'linear', u'format', u'zulu', u'also', u'available', u'dvd', u'also', u'recommendedzulu', u'dawn', u'1979', u'tells', u'complete', u'story', u'struggles', u'tribesmen', u'british', u'soldiers', u'commenced', u'1879', u'burt', u'lancaster', u'vhs', u'edition', u'dvd', u'editionparenthetical', u'number', u'preceding', u'title', u'1', u'10', u'viewer', u'poll', u'rating', u'found', u'film', u'resource', u'website', u'8', u'0', u'zulu', u'uk1964', u'sta

In [91]:
# We will use the spark ml library for this (took a bit long to figure out)
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams

# HashingTF calulates the 'tfid feature for us
#htf = HashingTF(numFeatures=2) # features need to match dimensions of output

"""
data = removeSW_output.zipWithIndex()

keys = removeSW_output.zipWithIndex().map(lambda data: data[1] )
hashingTF = HashingTF().setInputCol("words").setOutputCol("rawFeatures").setNumFeatures(numFeatures);
tf = hashingTF.transform(keys)


#... continue from the previous example
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)


tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)





// 4.) Create a IDFModel out of our flat vector RDD
        IDFModel idfModel = new IDF().fit(hashedData);
        // 5.) Create tfidf RDD
        JavaRDD<Vector> idf = idfModel.transform(hashedData);
        // 6.) Create Labledpoint RDD
        JavaRDD<LabeledPoint> idfTransformed = idf.zip(tupleData).map(t -> {
            return new LabeledPoint(t._2.label(), t._1);
        });
"""
############################

print "just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']"
print (htf.transform(["hi", "boo", "hi"]));

htf = HashingTF(numFeatures=200) 

hashedElements = removeSW_output.map(lambda x : htf.transform(x[1]))

labels = removeSW_output.map(lambda x : 1.0 if float(x[0]) >= 2.5 else 0.0)
#print hashedElements.top(2)
'''
[SparseVector(200, {1: 1.0, 32: 1.0, 45: 1.0, 90: 2.0, 94: 1.0, 124: 1.0, 126: 1.0, 135: 1.0, 138: 1.0, 185: 1.0, 199: 1.0}), SparseVector(200, {1: 2.0, 3: 1.0, 12: 2.0, 17: 1.0, 24: 1.0, 34: 1.0, 38: 1.0, 39: 1.0, 47: 2.0, 64: 1.0, 68: 2.0, 70: 1.0, 72: 1.0, 75: 1.0, 80: 1.0, 81: 1.0, 87: 1.0, 90: 1.0, 99: 1.0, 102: 1.0, 108: 1.0, 110: 1.0, 112: 1.0, 115: 1.0, 119: 1.0, 122: 1.0, 124: 1.0, 130: 1.0, 135: 1.0, 137: 1.0, 143: 1.0, 147: 1.0, 151: 1.0, 155: 1.0, 157: 1.0, 159: 2.0, 162: 1.0, 166: 1.0, 171: 1.0, 179: 1.0, 186: 1.0, 191: 1.0, 196: 1.0, 198: 1.0})]

'''

hashedPoints = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1]) ] if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])])))
#print hashedPoints.top(2)




hashedElements.cache()
idf = IDF().fit(hashedElements)
tfidf = idf.transform(hashedElements)


"""
IDF(minDocFreq=2).fit"""


#merge transformed data with original
merged=  labels.zip(tfidf)
print merged.top(2)
print 
feature_vector  = merged.map(lambda q: LabeledPoint(q[0], q[1])   )
print feature_vector.top(1)







#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
print feature_vector.filter(lambda x: x.label == 1.0).count()

just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']
(10,[3,7],[1.0,2.0])
[(1.0, SparseVector(200, {1: 1.9239, 3: 1.0044, 12: 1.2622, 17: 1.267, 24: 1.5998, 34: 1.0261, 38: 1.5115, 39: 1.0435, 47: 1.9026, 64: 1.2293, 68: 1.5112, 70: 1.3785, 72: 1.3311, 75: 1.3972, 80: 1.103, 81: 1.7388, 87: 1.144, 90: 0.7949, 99: 1.6047, 102: 1.5648, 108: 1.5172, 110: 1.1129, 112: 1.377, 115: 1.4915, 119: 1.4471, 122: 1.7743, 124: 1.1688, 130: 1.3666, 135: 0.724, 137: 1.3027, 143: 0.8674, 147: 0.7756, 151: 1.298, 155: 1.3738, 157: 1.556, 159: 3.0333, 162: 1.3837, 166: 1.4009, 171: 1.0717, 179: 1.57, 186: 0.9963, 191: 1.2454, 196: 1.2161, 198: 0.8439})), (1.0, SparseVector(200, {1: 0.962, 15: 2.1797, 19: 1.4782, 49: 1.5781, 53: 1.1388, 60: 1.0478, 61: 1.1023, 72: 1.3311, 74: 1.5207, 83: 1.598, 85: 1.6716, 101: 3.2041, 103: 0.8656, 132: 1.6989, 133: 1.6184, 173: 1.1847, 180: 1.517, 191: 1.2454}))]

[LabeledPoint(1.0, (200,[0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,2

In [92]:
# EXPLORE DIFFERENT ML APPROACHS
# https://spark.apache.org/docs/1.1.0/api/python/pyspark.mllib.classification-module.html

##  Trying SVM approach

In [93]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel, LogisticRegressionWithLBFGS 
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.feature import HashingTF, IDF


'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''


'''
dataToHash = removeSW_output.map(lambda (x,y): (x, " ".join(y)))
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=10) # features need to match dimensions of output

tf = htf.transform(dataToHash)

tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)


#print (tfidfIgnore.collect())
print 123

print ("SVM: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = tfidfIgnore #.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
*/
REFRESHED DATA
'''


svm_training_data, svm_testing_data= feature_vector.randomSplit([0.7, 0.3])


#LogisticRegressionWithLBFGS 
# Build the Support Vector Machine model
svm_model = SVMWithSGD.train(svm_training_data, iterations=500) # lower iterations so we arent here forever

# Evaluate the model on training data
svm_labelsAndPreds = svm_training_data.map(lambda p: (p.label, svm_model.predict(p.features)))
svm_trainErr = 100* svm_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(svm_training_data.count())
print("training error of the SVM model: " + "%.3f" %(svm_trainErr))
print("training accuracy of the SVM model: " + "%.3f" %(100-svm_trainErr))


#Evaluate on testing data.. the correct way lolz

svm_labelsAndPredsTest = svm_testing_data.map(lambda p: (p.label, svm_model.predict(p.features)))
svm_testErr = 100* svm_labelsAndPredsTest.filter(lambda (v, p): v != p).count() / float(svm_testing_data.count())
print("testing error of the SVM model: " + "%.3f" %(svm_testErr))
print("training accuracy of the SVM model: " + "%.3f" %(100-svm_testErr))






training error of the SVM model: 14.310
training accuracy of the SVM model: 85.690
testing error of the SVM model: 14.329
training accuracy of the SVM model: 85.671


In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS , LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

log_training_data, log_testing_data= feature_vector.randomSplit([0.6, 0.4])

#LogisticRegressionWithLBFGS 
# Build the Support Vector Machine model
log_model = LogisticRegressionWithLBFGS.train(log_training_data, iterations=500) # lower iterations so we arent here forever

# Evaluate the model on training data
log_labelsAndPreds = log_training_data.map(lambda p: (p.label, log_model.predict(p.features)))
log_trainErr = 100* log_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(log_training_data.count())
print "Log Training Accuracy: " + "%.3f" %(100-log_trainErr) + "%" 
print("training error of the LogisticRegressionWithLBFGS model= " + "%.3f" % (log_trainErr)) + "%" 

#Evaluate on testing data.. the correct way lolz

log_labelsAndPredsTest = log_testing_data.map(lambda p: (p.label, log_model.predict(p.features)))
log_testErr = 100* log_labelsAndPredsTest.filter(lambda (v, p): v != p).count() / float(log_testing_data.count())

print "Log Testing Accuracy: " + "%.3f" %(100-log_testErr) + "%" 
print("testing error of the LogisticRegressionWithLBFGS model: " + "%.3f" %(log_testErr)) + "%" 


NameError: name 'featurePV' is not defined

## Do some Predictions - Bayes

In [ ]:
### THIS USES NAIVE BAYES CLASSIFIER
### with 116 elements with the first output.txt 86KB , gets 80-93%
# tested with lots of data.... - 111MB

'''  REFRESH BLOCK
REFRESHING DATA
Re Run data -> refresh in case we are using another approach above this as well
Things to do - update number of features
'''
## Took forever to figure out
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams
htf = HashingTF(numFeatures=20 ) # features need to match dimensions of output
print ("BAYES: REFRESHING HTF, and FEATURE VECTOR")
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))
'''
REFRESHED DATA
'''

# would be nice to figure out which division is the best
#training_data, testing_data= feature_vector.randomSplit([0.6, 0.4])
bayes_training_data, bayes_testing_data= feature_vector.randomSplit([0.7, 0.3])

# parameters:
lamda = 1.0

# build Naive Bayes Classifier
nbay = mllib_class.NaiveBayes.train(bayes_training_data, lamda)

# Make prediction and test accuracy.




bayes_labelsAndPreds = bayes_training_data.map(lambda p : (nbay.predict(p.features), p.label))
#bayes_trainErr = bayes_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(bayes_training_data.count())
bayes_trainErr = 100.0 * bayes_labelsAndPreds.filter(lambda (x, v): x == v).count() / bayes_training_data.count()

print "Naive Bayes Training Accuracy: " + "%.3f" %(bayes_trainErr) + "%" 
print "Bayes Training error: " + "%.3f" % (100-bayes_trainErr) + "%" 


# Testing Data Accuracy

bayes_labelsAndPreds = bayes_testing_data.map(lambda p : (nbay.predict(p.features), p.label))
#bayes_testErr = bayes_labelsAndPreds.filter(lambda (v, p): v != p).count() / float(bayes_testing_data.count())
bayes_testErr = 100.0 * bayes_labelsAndPreds.filter(lambda (x, v): x == v).count() / bayes_testing_data.count()


print "Naive Bayes Testing Accuracy: " + "%.3f" %(bayes_testErr) + "%" 
print "Bayes Testing error: " + "%.3f" % (100-bayes_testErr) + "%" 

In [14]:
# PREDICTION - LETS USE OUR MODEL 


#newdata = sc.textFile("smaller_predictData.txt").map(lambda x: (cleanData(x).split('///')))
#newdata = sc.textFile("100k_parsed.txt").map(lambda x: (cleanData(x).split('///')))

# parse data for the params we want

strippedData = newdata.map(lambda L: (L[0], float(L[4]), L[6], L[7]) if len(L) == 8 else L )
completeData = strippedData.filter(lambda L: len(L) == 4)
print (strippedData.count())
print (completeData.count())
print (strippedData.count() == completeData.count())


movieData = completeData.map(lambda x: (  x[1], tokenize(x[3]))   )

#.filter(lambda x: float (x) >= 2.5).collect())
#print movieData.filter(lambda x: float(x[1]) >= 2.5).top(10)
#print movieData.filter(lambda x: float(x[1]) >= 2.5).count()


100000
99991
False


In [15]:
1.0 == 1.0

True

In [16]:

# HashingTF calulates the tfid feature for us
htf = HashingTF(numFeatures=1) # features need to match dimensions of output

featurePV = movieData.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))

print featurePV.filter(lambda x: x.label == 1.0).count()
print featurePV.count()


print featurePV.top(2)
#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
#print "Number of Positives: ", feature_vector.filter(lambda x: x.label == 1.0).count()
#print "Total Number of Data Points ", feature_vector.filter(lambda x: x.label == 1.0).count()


85876
99991
[LabeledPoint(1.0, [148.0]), LabeledPoint(1.0, [34.0])]


In [ ]:

sameModel = SVMModel.load(sc, output_dir)
predictionAndLabel = test.map(lambda p: (sameModel.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda (x, v): x == v).count() / test.count()

In [17]:
# SVM PREDICTIONS

print (svm_model) 
#print (log_model) 
#print (nbay)




svm_predictions = featurePV.map(lambda p: (p.label, svm_model.predict(p.features)))
svm_PErr = 100* svm_predictions.filter(lambda (v, p): v != p).count() / float(featurePV.count())
print "SVM Prediction Accuracy: " + "%.3f" %(svm_PErr) + "%" 
print "SVM  Prediction error: " + "%.3f" % (100-svm_PErr) + "%" 

featurePV = movieData.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))


(weights=[3.06841385062], intercept=0.0)
SVM Prediction Accuracy: 14.116%
SVM  Prediction error: 85.884%


In [23]:
# Log

log_predictions = featurePV.map(lambda p: (p.label, log_model.predict(p.features)))
log_PErr = 100* log_predictions.filter(lambda (v, p): v != p).count() / float(featurePV.count())
print "Log Regression Prediction Accuracy: " + "%.3f" %(log_PErr) + "%" 
print "Log Regression  Prediction error: " + "%.3f" % (100-log_PErr) + "%"

featurePV = movieData.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))


Log Regression Prediction Accuracy: 14.116%
Log Regression  Prediction error: 85.884%


In [20]:
# Bayes
bayes_predictions = featurePV.map(lambda p : (nbay.predict(p.features), p.label))
bayes_PErr = 100.0 * bayes_predictions.filter(lambda (x, v): x == v).count() / featurePV.count()
print "Naive Bayes Prediction Accuracy: " + "%.3f" %(bayes_PErr) + "%" 
print "Bayes Prediction error: " + "%.3f" % (100-bayes_PErr) + "%" 


Naive Bayes Prediction Accuracy: 85.884%
Bayes Prediction error: 14.116%


In [ ]:
'''

movies_txt2 = sc.textFile("50k_parsed.txt").map(lambda x: (cleanData(x).split('///')))



# take the data that we care most about
movies_new = movies_txt2.map(lambda L: (L[0], L[4], L[6], L[7]) if len(L) == 8 else L )
movies_new1 = movies_new.filter(lambda L: len(L) == 4) # lets just take all the data that has been parsed correctly

#print (movies_new.top(20))
print (movies_new1.count(), movies_new.count())
print (movies_new.count() == movies_new1.count())

removeHTMLTags = movies_new1.map(prepData) # remove html tags




removeSW_output= removeHTMLTags.map(lambda x: (  int((str(x[1])).replace("/","")), tokenize(x[3]))   )

# We will use the spark ml library for this (took a bit long to figure out)
## key notes, use mllib if you are using rdd, ditch ml if you are not using data frams

# HashingTF calulates the tfid feature for us
htf = HashingTF(numFeatures=1) # features need to match dimensions of output

print "just testing the hashing TF transformation on the array: ['hi', 'boo', 'hi']"
print (htf.transform(["hi", "boo", "hi"]));
feature_vector = removeSW_output.map(lambda elements: LabeledPoint(1.0, [htf.transform(elements[1])]) if float(elements[0]) >= 2.5 else LabeledPoint(0.0, [htf.transform(elements[1])]))

#feature_vector = removeSW3.map(lambda elements: LabeledPoint(1, [2]))
print feature_vector.filter(lambda x: x.label == 1.0).count()
'''
